In [1]:
##########################################################################
#
#
#
# THIS SCRIPT EXPANDS A GIVEN GENE SET TO DEFINE THE FULL SUBGRAPH THAT CONTAINS
#
# ALL NODES WITHIN THAT GENE SET BASED ON A RANDOM WALK.
#
#
#
#
#
##########################################################################


import os
import sys
import numpy as np
import scipy.stats as st
import networkx as nx
import itertools as it
import random as rd
import pymysql as mysql
import pickle 
#from fa2 import ForceAtlas2
import os.path
from networkx.utils import pairwise, not_implemented_for
from itertools import combinations, chain
from collections import defaultdict
import time
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
%matplotlib inline

#########################################################################
#
#          CALCULATES THE RANDOM WALK OPERATOR 
#             REQUIRES ADJACENCY MATRIX
#
#########################################################################

def rnd_walk_matrix(A, r, a, num_nodes):

    num = 1*num_nodes
    n = num_nodes
    factor = float((1-a)/n)
    E = np.multiply(factor,np.ones([n,n])) 
    A_tele = np.multiply(a,A) + E  

#     print(A_tele)
    M = normalize(A_tele, norm='l1', axis=0)                                 # column wise normalized MArkov matrix

    del A
    del A_tele
    del E

    U = np.identity(n,dtype=int) 
    H = (1-r)*M
    H1 = np.subtract(U,H)
    del U
    del M
    del H    

    W = r*np.linalg.inv(H1)   

    return W



#########################################################################
#
#          CONSTRUCTING STEINER TREE 
#
#########################################################################

def metric_closure(G, weight='weight'):
    """  Return the metric closure of a graph.

    The metric closure of a graph *G* is the complete graph in which each edge
    is weighted by the shortest path distance between the nodes in *G* .

    Parameters
    ----------
    G : NetworkX graph

    Returns
    -------
    NetworkX graph
        Metric closure of the graph `G`.

    """
    M = nx.Graph()

    Gnodes = set(G)

    # check for connected graph while processing first node
    all_paths_iter = nx.all_pairs_dijkstra(G, weight=weight)
    u, (distance, path) = next(all_paths_iter)
    if Gnodes - set(distance):
        msg = "G is not a connected graph. metric_closure is not defined."
        raise nx.NetworkXError(msg)
    Gnodes.remove(u)
    for v in Gnodes:
        M.add_edge(u, v, distance=distance[v], path=path[v])

    # first node done -- now process the rest
    for u, (distance, path) in all_paths_iter:
        Gnodes.remove(u)
        for v in Gnodes:
            M.add_edge(u, v, distance=distance[v], path=path[v])

    return M

def steiner_tree(G, terminal_nodes, weight='weight'):
    """ Return an approximation to the minimum Steiner tree of a graph.

    Parameters
    ----------
    G : NetworkX graph

    terminal_nodes : list
         A list of terminal nodes for which minimum steiner tree is
         to be found.

    Returns
    -------
    NetworkX graph
        Approximation to the minimum steiner tree of `G` induced by
        `terminal_nodes` .

    Notes
    -----
    Steiner tree can be approximated by computing the minimum spanning
    tree of the subgraph of the metric closure of the graph induced by the
    terminal nodes, where the metric closure of *G* is the complete graph in
    which each edge is weighted by the shortest path distance between the
    nodes in *G* .
    This algorithm produces a tree whose weight is within a (2 - (2 / t))
    factor of the weight of the optimal Steiner tree where *t* is number of
    terminal nodes.

    """
    # M is the subgraph of the metric closure induced by the terminal nodes of
    # G.
    M = metric_closure(G, weight=weight)
    # Use the 'distance' attribute of each edge provided by the metric closure
    # graph.
    H = M.subgraph(terminal_nodes)
    mst_edges = nx.minimum_spanning_edges(H, weight='distance', data=True)
    # Create an iterator over each edge in each shortest path; repeats are okay
    edges = chain.from_iterable(pairwise(d['path']) for u, v, d in mst_edges)
    T = G.edge_subgraph(edges)
    return T



#########################################################################
#########################################################################

def read_network_from_file(network_file):
    """
    Reads a network from an external file.

    * The edgelist must be provided as a tab-separated table. The
    first two columns of the table will be interpreted as an
    interaction gene1 <==> gene2

    * Lines that start with '#' will be ignored
    """

    G = nx.Graph()
    for line in open(network_file,'r'):
        # lines starting with '#' will be ignored
        if line[0]=='#':
            continue
        # The first two columns in the line will be interpreted as an
        # interaction gene1 <=> gene2
        line_data   = line.strip().split('\t')
        node1 = line_data[0]
        node2 = line_data[1]
        G.add_edge(node1,node2)

    print ("\n> done loading network:")
    print ("> network contains %s nodes and %s links" %(G.number_of_nodes(),
                                                       G.number_of_edges()))
    
    return G





In [5]:
# LOAD PPI 


G_ppi= read_network_from_file('mydata/Supplementary_File_1_interactome_edgelist_cleaned.txt')




> done loading network:
> network contains 18853 nodes and 483037 links


# RANDOM WALK MATRIX OF FULL NETWORK

In [6]:

# ██████╗  █████╗ ███╗   ██╗██████╗  ██████╗ ███╗   ███╗    ██╗    ██╗ █████╗ ██╗     ██╗  ██╗
# ██╔══██╗██╔══██╗████╗  ██║██╔══██╗██╔═══██╗████╗ ████║    ██║    ██║██╔══██╗██║     ██║ ██╔╝
# ██████╔╝███████║██╔██╗ ██║██║  ██║██║   ██║██╔████╔██║    ██║ █╗ ██║███████║██║     █████╔╝ 
# ██╔══██╗██╔══██║██║╚██╗██║██║  ██║██║   ██║██║╚██╔╝██║    ██║███╗██║██╔══██║██║     ██╔═██╗ 
# ██║  ██║██║  ██║██║ ╚████║██████╔╝╚██████╔╝██║ ╚═╝ ██║    ╚███╔███╔╝██║  ██║███████╗██║  ██╗
# ╚═╝  ╚═╝╚═╝  ╚═╝╚═╝  ╚═══╝╚═════╝  ╚═════╝ ╚═╝     ╚═╝     ╚══╝╚══╝ ╚═╝  ╚═╝╚══════╝╚═╝  ╚═╝
                                                                                            



t0 = time.time()

# RANDOM WALK PARAMETERS
r = .9
alpha = 1.
# infile = sys.argv[4]



# ###########################################################################
#                                                                           #
#    CALCULATES MATRIX INVERSION FOR GIVEN PARAMETERS                       #
#                                                                           #
                                                                            #
num_nodes = G_ppi.number_of_nodes()                                             #
A = nx.adjacency_matrix(G_ppi,sorted(G_ppi.nodes()))                                #
                                                                            #
#################################################################           #
#                                                                           #
#    INVERT MARKOV MATRIX & GENERATE RW MATRIX                              #
#                                                                           #
###################                                                         #
                                                                            #
W = rnd_walk_matrix(A, r, alpha, num_nodes)                                 #
                                                                            #
print('Inversion done')                                                     #
#################################################################           #
#                                                                           #
#    GENERATE DICT FOR NODE-LABELS TO INTEGERS                              #
#                                                                           #
###################                                                         #
                                                                            #
d_idx_entz = {}                                                             #
cc = 0                                                                      #
for entz in sorted(G_ppi.nodes()):                                              #
    d_idx_entz[cc] = entz                                                   #
    cc += 1                                                                 #
d_entz_idx = dict((y,x) for x,y in d_idx_entz.items())                      #
print('rnd walk computing time: %.2f' %float(time.time()-t0))               #
                                                                            #
#############################################################################





Inversion done
rnd walk computing time: 646.21


# LOAD SEED SETS

In [7]:
f = open('mydata/mono_AIM_AIF_IDs.txt','r')
lines = f.readlines()
f.close()
mono_all = lines[0].split('\n')[0].split('\t')[1:-1]


# RND WALK FROM SEEDs UNTIL GRAPH IS CONNECTED 

In [11]:
seedset_all = mono_all

# CHECK PPI OVERLAP

seedset_ppi = set(seedset_all) & set(G_ppi.nodes())


# PERFORM RANDOM WALK FROM SEED GENES
nodeset = []
for s_ent in seedset_ppi:
    nodeset.append(d_entz_idx[s_ent])

normprob = len(nodeset)
print('Size of initial set: %s seed genes' %normprob)
p0 = np.zeros(G_ppi.number_of_nodes())
# generate start vector
for n in range(len(p0)):
    if n in nodeset:
        p0[n] = 1.
        
pinf =  np.array(W.dot(p0))
   
# DICT WITH NODE ID AND PVIS 
d_n_p = {}
i = 0
for x in pinf:
#     print(i,x)
    d_n_p[d_idx_entz[i]] = x/normprob
    i += 1    
       
# # write file 
# outfile = 'ranked_genes_pvis.txt'
# f_out = open(outfile,'w')
# f_out.write('entrez_id,visit probab,cumulative\n')

pcum = 0
l_rndwnodes = []
cc = 0
for node, pvis in sorted(d_n_p.items(), key = lambda x: x[1], reverse = True):
    pcum += pvis
#     print(node,pvis,pcum)
    l_rndwnodes.append(node)
    G_test = G_ppi.subgraph(l_rndwnodes)
#     print(nx.is_connected(G_test))
    
    if nx.is_connected(G_test) and cc>len(seedset_ppi):
        break
#     del G_test

    cc += 1
print('# nodes: %s' %G_test.number_of_nodes())
print('# edges: %s' %G_test.number_of_edges())

nx.write_edgelist(G_test,'mydata/AutoCore.elist')

Size of initial set: 186 seed genes
# nodes: 399
# edges: 5244


NameError: name 'd_gene_terms_all' is not defined